In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
AAPL_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/AAPL_predictions.csv", index_col=0)


In [3]:
AAPL_predictions.head()

Date  Predictions
0  2019-06-28            1
1  2019-05-31            1
2  2019-04-30            1
3  2019-03-29            1
4  2019-02-28            1

In [4]:
AAPL_predictions.Date = pd.to_datetime(AAPL_predictions.Date, format="%Y-%m-%d")

In [5]:
AAPL_predictions['Predictions'] = AAPL_predictions['Predictions'].shift(-1)


In [6]:
AAPL_predictions = AAPL_predictions.dropna()


In [7]:
AAPL_predictions = AAPL_predictions[::-1]

In [8]:
AAPL_predictions.head(10)

Date  Predictions
111 2010-03-31          1.0
110 2010-04-30          0.0
109 2010-05-28          1.0
108 2010-06-30          1.0
107 2010-07-30          1.0
106 2010-08-31          1.0
105 2010-09-30          1.0
104 2010-10-29          1.0
103 2010-11-30          1.0
102 2010-12-31          1.0

In [9]:
AAPL_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/AAPL_monthly.csv", index_col =0)

In [10]:
AAPL_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
AAPL_predictions = AAPL_predictions.set_index("Date")

In [12]:
AAPL_monthly.head()

Open    High     Low   Close     Volume
2019-07-26  203.17  209.73  198.41  207.74  349033605
2019-06-28  175.60  201.57  170.27  197.92  515218768
2019-05-31  209.88  215.31  174.99  175.07  739456573
2019-04-30  191.64  208.48  188.38  200.67  506117812
2019-03-29  174.28  197.69  169.50  189.95  650981384

In [13]:
AAPL_final_df = pd.merge(AAPL_predictions, AAPL_monthly, right_index=True, left_index=True)

In [14]:
AAPL_final_df.index = pd.to_datetime(AAPL_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class BH(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.set_trailing_sl(4)
        self.start = self.data.index[0]
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
#         print (type(self.start))
        if self.data.index[-1] > self.start and (position.is_long!=True):
        
            self.buy()
        else:
            pass

In [18]:
bt = Backtest(AAPL_final_df, BH, cash=10000, commission=.01)
bt.run()



Start                     2010-03-31 00:00:00
End                       2019-06-28 00:00:00
Duration                   3376 days 00:00:00
Exposure [%]                                0
Equity Final [$]                      7427.24
Equity Peak [$]                       25034.1
Return [%]                           -25.7276
Buy & Hold Return [%]                 15.7787
Max. Drawdown [%]                    -86.0697
Avg. Drawdown [%]                    -5.06051
Max. Drawdown Duration      153 days 00:00:00
Avg. Drawdown Duration      117 days 00:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                            NaN
Max. Trade Duration                       NaT
Avg. Trade Duration                       NaT
Expectancy [%]                            NaN
SQN                                       NaN
Sharpe Ratio                      

In [19]:
bt.plot(superimpose=False, plot_volume=False)

In [ ]:
df = pd.DataFrame.from_dict([equity_dict])

In [ ]:
df = df.T

In [ ]:
df.columns = ['Equity']

In [ ]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/BH/AAPL_BH_Equity.csv")

In [ ]:
df